# PSL Coding Assignment 5

**Members**
-  Amy Hwang (ahwang22)
-  Christian Tam (cmtam2)
-  Monil Kaneria (kaneria2)

**Contributions:** 

Christian Tam worked on the following: review and testing 

Amy Hwang worked on the following: documentation and testing

Monil Kaneria worked on the following: functions, data processing, and testing

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

## Pegasos Algorithm

In [2]:
def pegasos_svm(X, y, lambda_param=0.01, epochs=20):
    # Initialize parameters
    n_samples, n_features = X.shape
    beta = np.zeros(n_features)
    alpha = 0
    t = 0

    for epoch in range(epochs):
        # Shuffle the data
        X, y = shuffle(X, y, random_state=epoch)

        for i in range(n_samples):
            t += 1
            eta_t = 1 / (t * lambda_param)
            
            # Calculate the gradient and update beta and alpha
            if y[i] * (np.dot(X[i].astype(float), beta) + alpha) < 1:
                beta = beta - eta_t * ((lambda_param * beta) - (y[i] * X[i].astype(float)))
                alpha = alpha + eta_t * y[i]
            else:
                beta = beta - (eta_t * lambda_param * beta)

    return beta, alpha

## Testing Code

In [3]:
def predict(X, beta, alpha):
    return np.where(np.dot(X, beta) + alpha > 0, 1, -1)

Load the datasets.

In [4]:
train_data = pd.read_csv('coding5_train.csv', header=0)
test_data = pd.read_csv('coding5_test.csv', header=0)

train_data = train_data.apply(pd.to_numeric, errors='coerce').dropna()
test_data = test_data.apply(pd.to_numeric, errors='coerce').dropna()

Split features and labels.

In [5]:
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values

Convert labels to +1 and -1.

In [6]:
y_train = np.where(y_train == 5, 1, -1)
y_test = np.where(y_test == 5, 1, -1)

Train the model with the Pegasos algorithm.

In [7]:
lambda_param = 0.2
beta, alpha = pegasos_svm(X_train, y_train, lambda_param=lambda_param, epochs=20)

Predictions.

In [8]:
train_predictions = predict(X_train, beta, alpha)
test_predictions = predict(X_test, beta, alpha)

## Confusion Matrices

In [9]:
def confusion_matrix(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == -1) & (y_pred == -1))
    FP = np.sum((y_true == -1) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == -1))
    return np.array([[TP, FN], [FP, TN]])

In [10]:
train_confusion = confusion_matrix(y_train, train_predictions)
test_confusion = confusion_matrix(y_test, test_predictions)

print("Confusion Matrix - Training Data:")
print(train_confusion)

print("Confusion Matrix - Test Data:")
print(test_confusion)

Confusion Matrix - Training Data:
[[100   0]
 [  0 100]]
Confusion Matrix - Test Data:
[[295   5]
 [ 15 285]]


## Test Error

In [11]:
test_error = np.mean(test_predictions != y_test)
print(f"Test Error: {test_error * 100:.2f}%")

Test Error: 3.33%
